# Scratchwork

This notebook is only used for trying out ideas.

In [1]:
import algebras as alg
import json
import os
import numpy as np
import itertools as it
# from pprint import pprint
import pprint as pp

# from itertools import combinations, permutations, product
import itertools as it

In [2]:
# Path to this repo
aa_path = os.path.join(os.getenv('PYPROJ'), 'abstract_algebra')

# Path to a directory containing Algebra definitions in JSON
alg_dir = os.path.join(aa_path, "Algebras")

## Groups for Testing

### D4 - Dihedral Group on 4 Vertices

In [3]:
d4_path = os.path.join(alg_dir, "d4_dihedral_group_on_4_vertices.json")
!cat {d4_path}

{"type": "Group",
 "name": "D_4",
 "description": "Dihedral group on four vertices",
 "element_names": ["e", "r", "r^2", "r^3", "f", "fr", "r^2f", "rf"],
 "alt_elem_names": ["()", "(0 1 2 3)", "(0 2)(1 3)", "(0 3 2 1)",
                    "(0 1)(2 3)", "(1 3)", "(0 3)(1 2)", "(0 2)"],
 "mult_table": [[0, 1, 2, 3, 4, 5, 6, 7],
                [1, 2, 3, 0, 7, 4, 5, 6],
                [2, 3, 0, 1, 6, 7, 4, 5],
                [3, 0, 1, 2, 5, 6, 7, 4],
                [4, 5, 6, 7, 0, 1, 2, 3],
                [5, 6, 7, 4, 3, 0, 1, 2],
                [6, 7, 4, 5, 2, 3, 0, 1],
                [7, 4, 5, 6, 1, 2, 3, 0]]
}

In [4]:
d4 = alg.Group(d4_path)
d4.about()


Group: D_4
Dihedral group on four vertices
Abelian? False
Elements:
   Index   Name   Inverse  Order
      0       e       e       1
      1       r     r^3       4
      2     r^2     r^2       2
      3     r^3       r       4
      4       f       f       2
      5      fr      fr       2
      6    r^2f    r^2f       2
      7      rf      rf       2
Cayley Table (showing indices):
[[0, 1, 2, 3, 4, 5, 6, 7],
 [1, 2, 3, 0, 7, 4, 5, 6],
 [2, 3, 0, 1, 6, 7, 4, 5],
 [3, 0, 1, 2, 5, 6, 7, 4],
 [4, 5, 6, 7, 0, 1, 2, 3],
 [5, 6, 7, 4, 3, 0, 1, 2],
 [6, 7, 4, 5, 2, 3, 0, 1],
 [7, 4, 5, 6, 1, 2, 3, 0]]


### V4 - Klein 4 Group

In [5]:
v4_path = os.path.join(alg_dir, "v4_klein_4_group.json")
!cat {v4_path}

{"type": "Group",
 "name": "V4",
 "description": "Klein-4 group",
 "element_names": ["e", "h", "v", "hv"],
 "mult_table": [[0, 1, 2, 3],
                [1, 0, 3, 2],
                [2, 3, 0, 1],
                [3, 2, 1, 0]]
}


In [6]:
v4 = alg.Group(v4_path)
v4.about()


Group: V4
Klein-4 group
Abelian? True
Elements:
   Index   Name   Inverse  Order
      0       e       e       1
      1       h       h       2
      2       v       v       2
      3      hv      hv       2
Cayley Table (showing indices):
[[0, 1, 2, 3], [1, 0, 3, 2], [2, 3, 0, 1], [3, 2, 1, 0]]


In [7]:
z2 = alg.generate_cyclic_group(2)
z2

Group('Z2',
'Autogenerated cyclic group of order 2',
['e', 'a'],
[[0, 1], [1, 0]]) 

In [8]:
z2_x_z2 = z2 * z2
z2_x_z2

Group('Z2_x_Z2',
'Direct product of Z2 & Z2',
['e:e', 'e:a', 'a:e', 'a:a'],
[[0, 1, 2, 3], [1, 0, 3, 2], [2, 3, 0, 1], [3, 2, 1, 0]]) 

In [9]:
z4 = alg.generate_cyclic_group(4)
z4

Group('Z4',
'Autogenerated cyclic group of order 4',
['e', 'a', 'a^2', 'a^3'],
[[0, 1, 2, 3], [1, 2, 3, 0], [2, 3, 0, 1], [3, 0, 1, 2]]) 

## Finite Rings

"Consider any finite set. Then its power set with respect to symmetric difference (as 1st operation) and set intersection (as 2nd operation) is an example of a finite ring." -- [Quora](https://www.quora.com/What-are-examples-of-finite-rings#:~:text=Matrices%20over%20any%20ring%20make,a%20finite%20ring%20with%20elements.)

### Powerset Functions

In [10]:
n = 20
ints_n = set(list(range(n)))
ints_n

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19}

[Source of powerset code](https://docs.python.org/3/library/itertools.html#itertools-recipes)

In [11]:
%time foo1 = [set(x) for x in list(alg.powerset(ints_n))]
len(foo1)

CPU times: user 2.03 s, sys: 238 ms, total: 2.27 s
Wall time: 2.28 s


1048576

### Set Operations

In [12]:
A = {0, 2, 4, 6, 8}
B = {1, 2, 3, 4, 5}

print(f"               Union: {A | B}")
print(f"        Intersection: {A & B}")
print(f"          Difference: {A - B}")
print(f"Symmetric Difference: {A ^ B}")

               Union: {0, 1, 2, 3, 4, 5, 6, 8}
        Intersection: {2, 4}
          Difference: {0, 8, 6}
Symmetric Difference: {0, 1, 3, 5, 6, 8}


TODO: Look for the paper by Carolyn Bean, "Group operations on the power set", Journal of Undergraduate Mathematics 8 #1 (March 1976), 13-17.

## Summarize Proper Subgroups of a Group

In [13]:
ps3 = alg.generate_powerset_group(3)
ps3.pprint()

print(f"\nIdentity Element: {ps3.identity}")
print(f"Abelian? {ps3.abelian()}\n")

ps3_prop_subs = ps3.proper_subgroups()
print(f"{ps3.name} has the following {len(ps3_prop_subs)} proper subgroups:\n")
for subgroup in ps3_prop_subs:
    subgroup.pprint()

Group('PS3',
'Autogenerated group on the powerset of 3 elements, with symmetric difference operator',
[(), (0,), (1,), (2,), (0, 1), (0, 2), (1, 2), (0, 1, 2)],
[[0, 1, 2, 3, 4, 5, 6, 7],
 [1, 0, 4, 5, 2, 3, 7, 6],
 [2, 4, 0, 6, 1, 7, 3, 5],
 [3, 5, 6, 0, 7, 1, 2, 4],
 [4, 2, 1, 7, 0, 6, 5, 3],
 [5, 3, 7, 1, 6, 0, 4, 2],
 [6, 7, 3, 2, 5, 4, 0, 1],
 [7, 6, 5, 4, 3, 2, 1, 0]]
)

Identity Element: ()
Abelian? True

PS3 has the following 14 proper subgroups:

Group('PS3_subgroup_0',
'Subgroup of: Autogenerated group on the powerset of 3 elements, with symmetric difference operator',
[(), (1, 2)],
[[0, 1], [1, 0]]
)
Group('PS3_subgroup_1',
'Subgroup of: Autogenerated group on the powerset of 3 elements, with symmetric difference operator',
[(), (2,), (0, 1), (0, 1, 2)],
[[0, 1, 2, 3], [1, 0, 3, 2], [2, 3, 0, 1], [3, 2, 1, 0]]
)
Group('PS3_subgroup_2',
'Subgroup of: Autogenerated group on the powerset of 3 elements, with symmetric difference operator',
[(), (0,)],
[[0, 1], [1, 0]]
)
Group('P

In [14]:
g0 = ps3_prop_subs[0]
gs = ps3_prop_subs[1:]
g0

Group('PS3_subgroup_0',
'Subgroup of: Autogenerated group on the powerset of 3 elements, with symmetric difference operator',
[(), (1, 2)],
[[0, 1], [1, 0]]) 

In [15]:
[g0.isomorphic(g) for g in gs]

[False,
 {(1, 2): (0,), (): ()},
 {(1, 2): (1,), (): ()},
 False,
 False,
 {(1, 2): (2,), (): ()},
 False,
 {(1, 2): (0, 2), (): ()},
 False,
 False,
 {(1, 2): (0, 1, 2), (): ()},
 {(1, 2): (0, 1), (): ()},
 False]

In [16]:
g1 = ps3_prop_subs[1]
gs2 = ps3_prop_subs[2:]
g1

Group('PS3_subgroup_1',
'Subgroup of: Autogenerated group on the powerset of 3 elements, with symmetric difference operator',
[(), (2,), (0, 1), (0, 1, 2)],
[[0, 1, 2, 3], [1, 0, 3, 2], [2, 3, 0, 1], [3, 2, 1, 0]]) 

In [17]:
[g1.isomorphic(g) for g in gs2]

[False,
 False,
 {(2,): (1,), (0, 1): (2,), (0, 1, 2): (1, 2), (): ()},
 {(2,): (0,), (0, 1): (2,), (0, 1, 2): (0, 2), (): ()},
 False,
 {(2,): (0,), (0, 1): (1,), (0, 1, 2): (0, 1), (): ()},
 False,
 {(2,): (0, 1), (0, 1): (0, 2), (0, 1, 2): (1, 2), (): ()},
 {(2,): (1,), (0, 1): (0, 2), (0, 1, 2): (0, 1, 2), (): ()},
 False,
 False,
 {(2,): (0,), (0, 1): (1, 2), (0, 1, 2): (0, 1, 2), (): ()}]